In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import pandas as pd

In [ ]:
# assumption: rows and columns ranges always start at 1
rows_interval = 8
rows_min = 8
rows_max = 128
rows = int(rows_max/rows_interval) - int(rows_min/rows_interval) + 1

columns_interval = 8
columns_min = 8
columns_max = 128
columns = int(columns_max/columns_interval) - int(columns_min/columns_interval) + 1

batch_count = 1000

operation = "op_n"
precision = 64

median_array = np.zeros((rows, columns))
master_array = np.zeros(100)

In [ ]:
# for n in n_range:
# load data
for i, n in enumerate(range(rows_min, rows_max + 1, rows_interval)):
    for j, m in enumerate(range(columns_min, columns_max + 1, columns_interval)):
        try:
            directoryname = f"blas_results/{operation}_{precision}/{n}_{m}_{batch_count}"
            filename = "results.csv"
            df = pd.read_csv(f"{directoryname}/{filename}", usecols=["BeginNs", "EndNs"])
            subtracted_df = df["EndNs"] - df["BeginNs"]
            subtracted_array = subtracted_df.to_numpy()
            median_array[i, j] = np.median(subtracted_array) / 1000
        except:
            print(n,m)
# directoryname = "blas_results/OP_N_64/10_10_100"
# filename = "results.csv"
# df = pd.read_csv(f"{directoryname}/{filename}", usecols=["BeginNs", "EndNs"])
# subtracted_df = df["EndNs"] - df["BeginNs"]
# subtracted_array = subtracted_df.to_numpy()
# subtracted_array = subtracted_array.transpose()
# master_array = np.vstack((master_array, subtracted_array))
# master_array = np.delete(master_array, 0, axis=0)

In [ ]:
print(master_array.shape)
print(master_array)

In [ ]:
fig, ax = plt.subplots(dpi=256)

c = ax.imshow(median_array, origin="lower")
plt.title(
    f"median time to perform gemvstridedbatched\n({operation}, {precision}-bit, batchCount={batch_count})"
)
plt.xlabel("Number of columns")
plt.ylabel("Number of rows")
cbar = plt.colorbar(c)
cbar.ax.set_ylabel("milliseconds")
plt.xticks([0, 32, 64, 96])
plt.yticks([0, 32, 64, 96])

plt.show()

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator

fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, dpi=256)

X = np.arange(rows_min, rows_max + 1)
Y = np.arange(columns_min, columns_max + 1)
X, Y = np.meshgrid(X, Y)

Z = median_array[X - 1, Y - 1]

surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False)
ax.view_init(elev=20.0, azim=-60)
plt.show()